# Introduction to the RockBlock and LayeredRockBlock objects


In [ ]:
import os
import numpy as np
import xarray as xr
import dask
%matplotlib inline

import holoviews as hv
hv.extension('bokeh', 'matplotlib')

from landlab import RasterModelGrid
from landlab.layers import RockBlock, LayeredRockBlock
from landlab.components import FlowAccumulator, FastscapeEroder
from landlab.plot import imshow_grid

In [ ]:
attrs = {'K_sp': {0: 0.0003,
                  1: 0.0001}}

mg = RasterModelGrid((50, 50), 100)
z = mg.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg.size('node'))
z += random_field - random_field.min()

z0s = 50 * np.arange(-20, 20)
z0s[-1] = z0s[-2] + 10000
ids = np.tile([0,1], 20) 

# Anticline
rb = LayeredRockBlock(mg, z0s, ids, x0=5000, y0=5000, function = lambda x, y : ((0.001*x)**2+(0.003*y)**2), attrs=attrs)

# Shallow dips
#rb = LayeredRockBlock(mg, z0s, ids, x0=5000, y0=5000, function = lambda x, y : ((0.001*x)+(0.003*y)), attrs=attrs)

# Steeper dips
#rb = LayeredRockBlock(mg, z0s, ids, x0=5000, y0=5000, function = lambda x, y : ((0.01*x)+(0.01*y)), attrs=attrs)

In [ ]:
imshow_grid(mg, 'K_sp')

In [ ]:
nts = 400
U = 0.001
dt = 1000

fa = FlowAccumulator(mg)
sp = FastscapeEroder(mg, K_sp='K_sp')

out_fields = ['topographic__elevation',
              'K_sp', 
              'rock_type__id']

ds = xr.Dataset(data_vars={'topographic__elevation' : (('y', 'x', 't'), np.empty((mg.shape[0], mg.shape[1], nts))),
                           'K_sp' : (('y', 'x', 't'), np.empty((mg.shape[0], mg.shape[1], nts))),
                           'rock_type__id': (('y', 'x', 't'), np.empty((mg.shape[0], mg.shape[1], nts)))},
                coords={'x': mg.x_of_node.reshape(mg.shape)[0,:],
                        'y': mg.y_of_node.reshape(mg.shape)[:, 1],
                        't': dt*np.arange(nts)})

In [ ]:
for i in range(nts):
    fa.run_one_step()
    sp.run_one_step(dt = dt)
    dz_ad = np.zeros(mg.size('node'))
    dz_ad[mg.core_nodes] = U * dt
    z += dz_ad
    rb.run_one_step(dz_advection = dz_ad)
    
    for of in out_fields:
        ds[of][:,:,i] = mg['node'][of].reshape(mg.shape)


In [ ]:
imshow_grid(mg, 'topographic__elevation')

In [ ]:
hvds = hv.Dataset(ds)
hvds

In [ ]:
%opts Image (cmap='viridis')

topo = hvds.to(hv.Image, ['x', 'y'], ['topographic__elevation'])
rock = hvds.to(hv.Image, ['x', 'y'], ['rock_type__id'])

topo + rock

Make inverted topography by filling RockBlock in with resistant material only where the DA is large. 

In [ ]:
mg2 = RasterModelGrid((50, 50), 100)
z2 = mg2.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg2.size('node'))
z2 += random_field - random_field.min()

thicknesses2 = [1000]
ids2 =[0]

attrs2 = {'K_sp': {0: 0.0001,
                   1: 0.00001}}

rb2 = RockBlock(mg2, thicknesses2, ids2, attrs=attrs2)

nts = 400
U = 0.001
dt = 1000

fa2 = FlowAccumulator(mg2)
sp2 = FastscapeEroder(mg2, K_sp='K_sp')

out_fields = ['topographic__elevation',
              'K_sp', 
              'rock_type__id']

ds2 = xr.Dataset(data_vars={'topographic__elevation' : (('y', 'x', 't'), np.empty((mg2.shape[0], mg2.shape[1], nts))),
                            'K_sp' : (('y', 'x', 't'), np.empty((mg2.shape[0], mg2.shape[1], nts))),
                            'rock_type__id': (('y', 'x', 't'), np.empty((mg2.shape[0], mg2.shape[1], nts)))},
                 coords={'x': mg2.x_of_node.reshape(mg2.shape)[0,:],
                         'y': mg2.y_of_node.reshape(mg2.shape)[:, 1],
                         't': dt*np.arange(nts)})
half_nts = int(nts/2)
for i in range(half_nts):
    fa2.run_one_step()
    sp2.run_one_step(dt = dt)
    dz_ad2 = np.zeros(mg2.size('node'))
    dz_ad2[mg2.core_nodes] = U * dt
    z2 += dz_ad2
    rb2.run_one_step(dz_advection = dz_ad2)
    
    for of in out_fields:
        ds2[of][:,:,i] = mg2['node'][of].reshape(mg2.shape)


In [ ]:
imshow_grid(mg2, 'topographic__elevation')

In [ ]:
volcano_deposits = np.zeros(mg2.size('node'))
da_big_enough = mg2['node']['drainage_area']>1e4
volcano_deposits[da_big_enough] = 30.0
volcano_deposits[mg2.boundary_nodes] = 0.0
rb2.add_layer(volcano_deposits, rock_id=1)

imshow_grid(mg2, da_big_enough)

In [ ]:
for i in range(half_nts, nts):
    fa2.run_one_step()
    sp2.run_one_step(dt = dt)
    dz_ad = np.zeros(mg2.size('node'))
    dz_ad[mg2.core_nodes] = U * dt
    z2 += dz_ad
    rb2.run_one_step(dz_advection = dz_ad)
    
    for of in out_fields:
        ds2[of][:,:,i] = mg2['node'][of].reshape(mg2.shape)

In [ ]:
imshow_grid(mg2, 'topographic__elevation')

In [ ]:
%opts Image (cmap='viridis')

hvds2 = hv.Dataset(ds2)
topo2 = hvds2.to(hv.Image, ['x', 'y'], ['topographic__elevation'])
rock2 = hvds2.to(hv.Image, ['x', 'y'], ['rock_type__id'])

topo2 + rock2